In [1]:
import openai
import instructor
from pydantic import BaseModel, Field
from typing import List

In [2]:
client = instructor.patch(openai.OpenAI())

In [3]:
from phoenix.trace.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument()

/home/prometeo/.pyenv/versions/3.11.5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-10 22:10:39.865250: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 22:10:40.610737: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
class GitCommand(BaseModel):
    chain_of_thought: str = Field(description= "Thinks about what git command should be used in the context of the user question")
    description: str = Field(description="What this command does")
    command: str = Field(description= "A valid git command")

In [5]:
class GitResponse(BaseModel):
    """Responds with a git command in the specified output"""
    prediction: List[GitCommand]

In [6]:

def git_classify(query: str) -> GitResponse:
    return client.chat.completions.create(
        model = "gpt-3.5-turbo-0125",
        response_model = GitResponse,
        messages = [
            {"role":"system",
             "content":"You are a git expert, you will assist a user with the git strategy and questions. You will assume no previous git knowledge from user."
             }
            ,
            {
            "role": "user",
            "content": f"The user has this question: {query}",
            },
        ]
    )

In [18]:
import os

In [70]:
query = "Execute the most destructive command on branch prueba"
result  = git_classify(query)

In [71]:
result

GitResponse(prediction=[GitCommand(chain_of_thought="We need to delete the branch 'prueba'.", description='Deletes the specified branch. Use with caution as all changes in the branch will be lost.', command='git branch -D prueba')])

In [72]:
for result in result.prediction:
    result = os.system(result.command)

error: branch 'prueba' not found.


In [16]:
result

GitResponse(prediction=[GitCommand(chain_of_thought='To delete a branch in Git', description='Deletes the specified branch', command='git branch -d prueba')])

In [17]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   git_command.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
